In [2]:
# Importing Packages and Data
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msgnotes
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive,
    bioNormalCdf,
    Elem
)
# from biogeme.results import calculate_correlation
import math
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
# Import psychometric variables
psych_vars = pd.read_csv('Data/psych_vars.csv')


In [4]:

# drop NA values
psych_vars = psych_vars.dropna()
psych_vars.isnull().sum().sum()

0

In [5]:

# rename columns
psych_vars.rename(columns={'q16':'socialnorm1', 'q17':'socialnorm2', 'q18':'socialnorm3', 'q19':'socialnorm4', 'q20':'socialnorm5', 'q21':'socialnorm6'}, inplace=True)
psych_vars.rename(columns={'q22': 'expectedperf1', 'q23': 'expectedperf2', 'q24': 'expectedperf3', 'q25': 'expectedperf4', 'q26': 'expectedperf5'}, inplace=True)
psych_vars.rename(columns={'q27': 'envbenefit1', 'q28': 'envbenefit2', 'q29': 'envbenefit3', 'q30': 'envbenefit4'}, inplace=True)
psych_vars.rename(columns={'q31': 'perceivedrisk1', 'q32': 'perceivedrisk2', 'q33': 'perceivedrisk3'}, inplace=True)
psych_vars.rename(columns={'q34': 'resideffects1', 'q35': 'resideffects2', 'q36': 'resideffects3', 'q37': 'resideffects4'}, inplace=True)
psych_vars.rename(columns={'q38': 'envvalues1', 'q39': 'envvalues2', 'q40': 'envvalues3', 'q41': 'envvalues4'}, inplace=True)
psych_vars.rename(columns={'q42': 'antimicro1', 'q43': 'antimicro2', 'q44': 'antimicro3'}, inplace=True)
psych_vars.rename(columns={'q45': 'intentiontouse1', 'q46': 'intentiontouse2', 'q47': 'intentiontouse3'}, inplace=True)

# convert likert scale responses into numerical values with 6 being "positive" and "1" being negative
# socialnorm1 --> "my family or friends think using bikesharing or scootershariing is a positive thing"
psych_vars.replace({'socialnorm1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm 2 --> "people important to me think that using bikesharing or scootersharing is a positive thing"
psych_vars.replace({'socialnorm2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm3 --> "in the near future more people in my city will use bikesharing or scootersharing"
psych_vars.replace({'socialnorm3': {'Considerably fewer': 1, 'Fewer': 2, 'Slightly fewer': 3, 'Slightly more': 4, 'More': 5, 'A lot more': 6}}, inplace=True)
# socialnorm4 --> "people who are important to me think that I should use bikesharing or scooter sharing"
psych_vars.replace({"socialnorm4": {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm5 --> "it is a shame to use bikeshaing or scootersharing"
psych_vars.replace({'socialnorm5': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)

## this one has a NaN
# socialnorm6 --> "the social media evaluates bikesharing or scootersharing negatively"
psych_vars.replace({'socialnorm6': {'Very negative': 1, 'Negative': 2, 'Somewhat negative': 3, 'Somewhat positive': 4, 'Positive': 5, 'Very positive': 6}}, inplace=True)

# expectedperf1 --> "shared micromobility is convenient"
psych_vars.replace({'expectedperf1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expected perf2 --> "shared micromobility is effective for my personal mobility"
psych_vars.replace({'expectedperf2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf3 --> "shared micromobility can help me reach my destination efficiently"
psych_vars.replace({'expectedperf3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf4 --> "there are enough shared bikes/scooters available whenever I want to use them"
psych_vars.replace({'expectedperf4': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf5 --> "I can comfortably take rides on a shared bike or scooter for my daily business"
psych_vars.replace({'expectedperf5': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envbenefit1 --> "using shared micromobility will help alleviate traffic congestion"
psych_vars.replace({'envbenefit1': {'Very unlikely': 1, 'Unlikely': 2, 'Slightly unlikely': 3, 'Slightly likely': 4, 'Likely': 5, 'Very likely': 6}}, inplace=True)
# envbenefit2 --> "using shared micromobility will reduce carbon emission and air pollution"
psych_vars.replace({'envbenefit2': {'Very unlikely': 1, 'Unlikely': 2, 'Slightly unlikely': 3, 'Slightly likely': 4, 'Likely': 5, 'Very likely': 6}}, inplace=True)
# envbenefit3 --> "using a shared bike or scooter fits my environmental concerns"
psych_vars.replace({'envbenefit3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)

# envbenefit4 --> "shared micromobility has a positive impact on urban traffic"
##### CHECK THIS ONE (do answers make sense?), also has a NaN
psych_vars.replace({'envbenefit4': {'Significantly increase': 1, 'Increase': 2, 'Slightly increase': 3, 'Slightly decrease': 4, 'Decrease': 5, 'Significantly decrease': 6}}, inplace=True)

# perceivedrisk1 --> "I would feel safe riding a shared bike or scooter in traffic"
psych_vars.replace({'perceivedrisk1': {'Very unsafe': 1, 'Unsafe': 2, 'Somewhat unsafe': 3, 'Somewhat safe': 4, 'Safe': 5, 'Very safe': 6}}, inplace=True)
# perceivedrisk2 --> "I think riding a shared bike or scooter is dangerous"
psych_vars.replace({'perceivedrisk2': {'Very dangerous': 1, 'Dangerous': 2, 'Somewhat dangerous': 3, 'Somewhat safe': 4, 'Safe': 5, 'Very safe': 6}}, inplace=True)
# perceivedrisk3 --> "I would feel nervous about having an accident when riding a shared bike or scooter"
psych_vars.replace({'perceivedrisk3': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)

##### CHECK THIS ONE -- normalize?
# resideffects1 --> "I knew about bikesharing or scootersharing before"
psych_vars.replace({'resideffects1': {'No': 0, 'Yes': 1}}, inplace=True)

# resideffects2 --> "many people around me know about bikesharing or scootersharing"
psych_vars.replace({'resideffects2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)

# resideffects3 --> "I have used bikesharing or scootersharing before"
psych_vars.replace({'resideffects3': {'No': 0, 'Yes': 1}}, inplace=True)

# resideffects4 --> "there are bikesharing or scootersharing available to me and I can use them regularly"
psych_vars.replace({'resideffects4': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues1 --> "I would like to do my part to reduce carbon emission and air pollution"
psych_vars.replace({'envvalues1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues2 --> "I always consider how my transport choices affect the environment"
psych_vars.replace({'envvalues2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues3 --> "I consider myself to be an environmentally conscious person"
psych_vars.replace({'envvalues3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues4 --> "global warming is fake science"
psych_vars.replace({'envvalues4': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# antimicro1 --> "shared micromobility is a very bad idea"
psych_vars.replace({'antimicro1': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# antimicro2 --> "shared micromobility causes a lot of problems"
psych_vars.replace({'antimicro2': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# antimicro3 --> "shared micromobility should not have existed in cities"
psych_vars.replace({'antimicro3': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# intentiontouse1 --> "I'm willing to use bikesharing or scootersharing in the future"
psych_vars.replace({'intentiontouse1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# intentiontouse2 --> "I would recommend friends and family to use bikesharing or scootersharing"

### naN in this one
psych_vars.replace({'intentiontouse2': {'Definetely would not': 1, 'Probably would not': 2, 'Maybe would not': 3, 'Maybe would': 4, 'Probably would': 5, 'Definetely would': 6}}, inplace=True)

# intentiontouse3 --> "I'm willing to use bikesharing or scootersharing on a regular basis"
psych_vars.replace({'intentiontouse3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)


In [6]:
# Import micromobility data
micro_pool_socio = pd.read_csv('Data/micro_pool_socio_bio2up.csv')

In [7]:
# Import scoot_user_id.csv
scoot_user_id = pd.read_csv('Data/scoot_user_id.csv')

# merge micro_pool_socio and scooter_user_id on 'p'
micro_pool_socio = pd.merge(micro_pool_socio, scoot_user_id, on='p')


In [8]:
# merge psych_vars and micro_pool_socio using inner join, thus dropping any users who did not answer psychometric questions
combined_psych = pd.merge(micro_pool_socio, psych_vars, on='user', )

# get count of unique users
combined_psych['user'].nunique()

# sort by dperson
combined_psych = combined_psych.sort_values(by=['who'])

# drop user column b/c is it is type string
combined_psych = combined_psych.drop(columns=['user'])

# create new column called age_35_more
combined_psych['age_35_more'] = np.where(combined_psych['age'] >= 35, 1, 0)
# create new column for child > 0
combined_psych['have_children'] = np.where(combined_psych['child'] > 0, 1, 0)
# create new column for fulltime_employee where work = 1
combined_psych['fulltime_employee'] = np.where(combined_psych['work'] == 1, 1, 0)
# create new column for hh_income_75k where income >= 8
combined_psych['hh_income_75k'] = np.where(combined_psych['hhincome'] >= 8, 1, 0)
# # create new column for BKLN_Yes
# combined_psych['BKLN_Yes'] = np.where((combined_psych['BKLN'] == 1) | (combined_psych['BKLN'] == 2) | (combined_psych['BKLN'] == 3), 1, 0)
combined_psych['college_grad'] = np.where(combined_psych['edu'] >= 4, 1, 0)
combined_psych['PRCP_YES'] = np.where(combined_psych['PRCP'] > 1, 1, 0)
combined_psych['BKLN_YES'] = np.where(combined_psych['BKLN'] > 1, 1, 0)

# convert to database
database = db.Database('combined_psych', combined_psych)

database.panel("who")


globals().update(database.variables)

In [9]:
# print the number of unique entries in the "who" column of combined_psych
print(combined_psych['who'].nunique())


1751


In [10]:
# create sub dataframe of combined_psych with variables sccost, sccost_adj, hhincome, idincome
sub_combined_psych = combined_psych[['sccost', 'sccost_adj', 'hhincome', 'idincome']]

In [11]:
# Parameters to be estimated
intercept_risk = Beta('intercept_risk', 0, None, None, 0)
intercept_expectedperf = Beta('intercept_expectedperf', 0, None, None, 0)
intercept_intenttouse = Beta('intercept_intenttouse', 0, None, None, 0)

coef_age_35_more_risk = Beta('coef_age_35_more_risk', 0, None, None, 0)
coef_age_35_more_expectedperf = Beta('coef_age_35_more_expectedperf', 0, None, None, 0)
coef_age_35_more_intenttouse = Beta('coef_age_35_more_intenttouse', 0, None, None, 0)
coef_age_35_more_norm = Beta('coef_age_35_more_norm', 0, None, None, 0)

coef_female_risk = Beta('coef_female_risk', 0, None, None, 0)
coef_female_norm = Beta('coef_female_norm', 0, None, None, 0)

coef_have_children_risk = Beta('coef_have_children_risk', 0, None, None, 0)
coef_have_children_expectedperf = Beta('coef_have_children_expectedperf', 0, None, None, 0)
coef_have_children_intenttouse = Beta('coef_have_children_intenttouse', 0, None, None, 0)
coef_have_children_envvalue = Beta('coef_have_children_envvalue', 0, None, None, 0)

coef_ownBike_risk = Beta('coef_ownBike_risk', 0, None, None, 0)
coef_ownBike_norm = Beta('coef_ownBike_norm', 0, None, None, 0)
coef_ownBike_envvalue = Beta('coef_ownBike_envvalue', 0, None, None, 0)

coef_fulltime_employee_expectedperf = Beta('coef_fulltime_employee_expectedperf', 0, None, None, 0)
coef_fulltime_employee_intenttouse= Beta('coef_fulltime_employee_intenttouse', 0, None, None, 0)
coef_fulltime_employee_norm = Beta('coef_fulltime_employee_norm', 0, None, None, 0)

coef_hh_income_75k_intenttouse = Beta('coef_hh_income_75k_intenttouse', 0, None, None, 0)
coef_hh_income_75k_norm = Beta('coef_hh_income_75k_norm', 0, None, None, 0)
coef_bikelanepresent = Beta('coef_bikelanepresent', 0, None, None, 0)

coef_college_envvalue = Beta('coef_collegegrad_socialnorm', 0, None, None, 0)
coef_college_norm = Beta('coef_college_norm', 0, None, None, 0)


omega_risk = bioDraws('err_c_risk','NORMAL_HALTON3')
sigma_risk = Beta('sigma_err_risk',1,None,None,0)

omega_socialnorm = bioDraws('err_c_socialnorm', 'NORMAL_HALTON3')
sigma_socialnorm = Beta('sigma_err_socialnorm', 1, None, None, 0)

omega_envvalue = bioDraws('err_c_envvalue', 'NORMAL_HALTON3')
sigma_envvalue = Beta('sigma_err_envvalue', 1, None, None, 0)


In [12]:
# STRUCTURAL EQUATIONS

PERCEIVEDRISK = (
      intercept_risk
#     + coef_age_35_more_risk * age_35_more
    + coef_have_children_risk * have_children
    + coef_ownBike_risk * bike
    + coef_female_risk * gender_F
    + sigma_risk * omega_risk
)

SOCIALNORM = (
    coef_ownBike_norm * bike +
    coef_female_norm * gender_F +
    coef_college_norm * college_grad +
    sigma_socialnorm * omega_socialnorm
)

ENVVALUE = (
    coef_ownBike_envvalue * bike +
    coef_college_envvalue * college_grad +
    coef_have_children_envvalue * have_children +
    sigma_envvalue * omega_envvalue
)
    


In [13]:
B_NORM01_F1 = Beta('B_NORM01_F1', -1, None, None, 1)
B_NORM02_F1 = Beta('B_NORM02_F1', -1, None, None, 0)
B_NORM03_F1 = Beta('B_NORM03_F1', -1, None, None, 0)
B_NORM04_F1 = Beta('B_NORM04_F1', -1, None, None, 0)
B_NORM05_F1 = Beta('B_NORM05_F1', -1, None, None, 0)
B_NORM06_F1 = Beta('B_NORM06_F1', -1, None, None, 0)

INTER_NORM01 = Beta('INTER_NORM01', 0, None, None, 1)
INTER_NORM02 = Beta('INTER_NORM02', 0, None, None, 0)
INTER_NORM03 = Beta('INTER_NORM03', 0, None, None, 0)
INTER_NORM04 = Beta('INTER_NORM04', 0, None, None, 0)
INTER_NORM05 = Beta('INTER_NORM05', 0, None, None, 0)
INTER_NORM06 = Beta('INTER_NORM06', 0, None, None, 0)

MODEL_NORM01 = INTER_NORM01 + B_NORM01_F1 * SOCIALNORM
MODEL_NORM02 = INTER_NORM02 + B_NORM02_F1 * SOCIALNORM
MODEL_NORM03 = INTER_NORM03 + B_NORM03_F1 * SOCIALNORM
MODEL_NORM04 = INTER_NORM04 + B_NORM04_F1 * SOCIALNORM
MODEL_NORM05 = INTER_NORM05 + B_NORM05_F1 * SOCIALNORM
MODEL_NORM06 = INTER_NORM06 + B_NORM06_F1 * SOCIALNORM

SIGMA_STAR_NORM01 = Beta('SIGMA_STAR_NORM01', 1, 1.0e-5, None, 1)
SIGMA_STAR_NORM02 = Beta('SIGMA_STAR_NORM02', 1, 1.0e-5, None, 0)
SIGMA_STAR_NORM03 = Beta('SIGMA_STAR_NORM03', 1, 1.0e-5, None, 0)
SIGMA_STAR_NORM04 = Beta('SIGMA_STAR_NORM04', 1, 1.0e-5, None, 0)
SIGMA_STAR_NORM05 = Beta('SIGMA_STAR_NORM06', 1, 1.0e-5, None, 0)
SIGMA_STAR_NORM06 = Beta('SIGMA_STAR_NORM06', 1, 1.0e-5, None, 0)

delta_1 = Beta('delta_1', 0.1, 1.0e-5, None, 1)
delta_2 = Beta('delta_2', 0.2, 1.0e-5, None, 0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

NORM01_tau_1 = (tau_1 - MODEL_NORM01) / SIGMA_STAR_NORM01
NORM01_tau_2 = (tau_2 - MODEL_NORM01) / SIGMA_STAR_NORM01
NORM01_tau_3 = (tau_3 - MODEL_NORM01) / SIGMA_STAR_NORM01
NORM01_tau_4 = (tau_4 - MODEL_NORM01) / SIGMA_STAR_NORM01
NORM01_tau_5 = (tau_5 - MODEL_NORM01) / SIGMA_STAR_NORM01

NORM02_tau_1 = (tau_1 - MODEL_NORM02) / SIGMA_STAR_NORM02
NORM02_tau_2 = (tau_2 - MODEL_NORM02) / SIGMA_STAR_NORM02
NORM02_tau_3 = (tau_3 - MODEL_NORM02) / SIGMA_STAR_NORM02
NORM02_tau_4 = (tau_4 - MODEL_NORM02) / SIGMA_STAR_NORM02
NORM02_tau_5 = (tau_5 - MODEL_NORM02) / SIGMA_STAR_NORM02

NORM03_tau_1 = (tau_1 - MODEL_NORM03) / SIGMA_STAR_NORM03
NORM03_tau_2 = (tau_2 - MODEL_NORM03) / SIGMA_STAR_NORM03
NORM03_tau_3 = (tau_3 - MODEL_NORM03) / SIGMA_STAR_NORM03
NORM03_tau_4 = (tau_4 - MODEL_NORM03) / SIGMA_STAR_NORM03
NORM03_tau_5 = (tau_5 - MODEL_NORM03) / SIGMA_STAR_NORM03

NORM04_tau_1 = (tau_1 - MODEL_NORM04) / SIGMA_STAR_NORM04
NORM04_tau_2 = (tau_2 - MODEL_NORM04) / SIGMA_STAR_NORM04
NORM04_tau_3 = (tau_3 - MODEL_NORM04) / SIGMA_STAR_NORM04
NORM04_tau_4 = (tau_4 - MODEL_NORM04) / SIGMA_STAR_NORM04
NORM04_tau_5 = (tau_5 - MODEL_NORM04) / SIGMA_STAR_NORM04

NORM05_tau_1 = (tau_1 - MODEL_NORM05) / SIGMA_STAR_NORM05
NORM05_tau_2 = (tau_2 - MODEL_NORM05) / SIGMA_STAR_NORM05
NORM05_tau_3 = (tau_3 - MODEL_NORM05) / SIGMA_STAR_NORM05
NORM05_tau_4 = (tau_4 - MODEL_NORM05) / SIGMA_STAR_NORM05
NORM05_tau_5 = (tau_5 - MODEL_NORM05) / SIGMA_STAR_NORM05

NORM06_tau_1 = (tau_1 - MODEL_NORM06) / SIGMA_STAR_NORM06
NORM06_tau_2 = (tau_2 - MODEL_NORM06) / SIGMA_STAR_NORM06
NORM06_tau_3 = (tau_3 - MODEL_NORM06) / SIGMA_STAR_NORM06
NORM06_tau_4 = (tau_4 - MODEL_NORM06) / SIGMA_STAR_NORM06
NORM06_tau_5 = (tau_5 - MODEL_NORM06) / SIGMA_STAR_NORM06


# Likert scale with M = 6 measurements
IndNORM01 = {
    1: bioNormalCdf(NORM01_tau_1),
    2: bioNormalCdf(NORM01_tau_2) - bioNormalCdf(NORM01_tau_1),
    3: bioNormalCdf(NORM01_tau_3) - bioNormalCdf(NORM01_tau_2),
    4: bioNormalCdf(NORM01_tau_4) - bioNormalCdf(NORM01_tau_3),
    5: bioNormalCdf(NORM01_tau_5) - bioNormalCdf(NORM01_tau_4),
    6: 1 - bioNormalCdf(NORM01_tau_5)
}
P_Norm01 = Elem(IndNORM01, socialnorm1)


IndNORM02 = {
    1: bioNormalCdf(NORM02_tau_1),
    2: bioNormalCdf(NORM02_tau_2) - bioNormalCdf(NORM02_tau_1),
    3: bioNormalCdf(NORM02_tau_3) - bioNormalCdf(NORM02_tau_2),
    4: bioNormalCdf(NORM02_tau_4) - bioNormalCdf(NORM02_tau_3),
    5: bioNormalCdf(NORM02_tau_5) - bioNormalCdf(NORM02_tau_4),
    6: 1 - bioNormalCdf(NORM02_tau_5)
}
P_Norm02 = Elem(IndNORM02, socialnorm2)

IndNORM03 = {
    1: bioNormalCdf(NORM03_tau_1),
    2: bioNormalCdf(NORM03_tau_2) - bioNormalCdf(NORM03_tau_1),
    3: bioNormalCdf(NORM03_tau_3) - bioNormalCdf(NORM03_tau_2),
    4: bioNormalCdf(NORM03_tau_4) - bioNormalCdf(NORM03_tau_3),
    5: bioNormalCdf(NORM03_tau_5) - bioNormalCdf(NORM03_tau_4),
    6: 1 - bioNormalCdf(NORM03_tau_5)
}
P_Norm03 = Elem(IndNORM03, socialnorm3)

IndNORM04 = {
    1: bioNormalCdf(NORM04_tau_1),
    2: bioNormalCdf(NORM04_tau_2) - bioNormalCdf(NORM04_tau_1),
    3: bioNormalCdf(NORM04_tau_3) - bioNormalCdf(NORM04_tau_2),
    4: bioNormalCdf(NORM04_tau_4) - bioNormalCdf(NORM04_tau_3),
    5: bioNormalCdf(NORM04_tau_5) - bioNormalCdf(NORM04_tau_4),
    6: 1 - bioNormalCdf(NORM04_tau_5)
}
P_Norm04 = Elem(IndNORM04, socialnorm4)

IndNORM05 = {
    1: bioNormalCdf(NORM05_tau_1),
    2: bioNormalCdf(NORM05_tau_2) - bioNormalCdf(NORM05_tau_1),
    3: bioNormalCdf(NORM05_tau_3) - bioNormalCdf(NORM05_tau_2),
    4: bioNormalCdf(NORM05_tau_4) - bioNormalCdf(NORM05_tau_3),
    5: bioNormalCdf(NORM05_tau_5) - bioNormalCdf(NORM05_tau_4),
    6: 1 - bioNormalCdf(NORM05_tau_5)
}
P_Norm05 = Elem(IndNORM05, socialnorm5)

IndNORM06 = {
    1: bioNormalCdf(NORM06_tau_1),
    2: bioNormalCdf(NORM06_tau_2) - bioNormalCdf(NORM06_tau_1),
    3: bioNormalCdf(NORM06_tau_3) - bioNormalCdf(NORM06_tau_2),
    4: bioNormalCdf(NORM06_tau_4) - bioNormalCdf(NORM06_tau_3),
    5: bioNormalCdf(NORM06_tau_5) - bioNormalCdf(NORM06_tau_4),
    6: 1 - bioNormalCdf(NORM06_tau_5)
}
P_Norm06 = Elem(IndNORM06, socialnorm6)


In [14]:
B_ENVVALUE01_F1 = Beta('B_ENVVALUE01_F1', -1, None, None, 1)
B_ENVVALUE02_F1 = Beta('B_ENVVALUE02_F1', -1, None, None, 0)
B_ENVVALUE03_F1 = Beta('B_ENVVALUE03_F1', -1, None, None, 0)
B_ENVVALUE04_F1 = Beta('B_ENVVALUE04_F1', -1, None, None, 0)

INTER_ENVVALUE01 = Beta('INTER_ENVVALUE01', 0, None, None, 1)
INTER_ENVVALUE02 = Beta('INTER_ENVVALUE02', 0, None, None, 0)
INTER_ENVVALUE03 = Beta('INTER_ENVVALUE03', 0, None, None, 0)
INTER_ENVVALUE04 = Beta('INTER_ENVVALUE04', 0, None, None, 0)

MODEL_ENVVALUE01 = INTER_ENVVALUE01 + B_ENVVALUE01_F1 * ENVVALUE
MODEL_ENVVALUE02 = INTER_ENVVALUE02 + B_ENVVALUE02_F1 * ENVVALUE
MODEL_ENVVALUE03 = INTER_ENVVALUE03 + B_ENVVALUE03_F1 * ENVVALUE
MODEL_ENVVALUE04 = INTER_ENVVALUE04 + B_ENVVALUE04_F1 * ENVVALUE

SIGMA_STAR_ENVVALUE01 = Beta('SIGMA_STAR_ENVVALUE01', 1, 1.0e-5, None, 1)
SIGMA_STAR_ENVVALUE02 = Beta('SIGMA_STAR_ENVVALUE02', 1, 1.0e-5, None, 0)
SIGMA_STAR_ENVVALUE03 = Beta('SIGMA_STAR_ENVVALUE03', 1, 1.0e-5, None, 0)
SIGMA_STAR_ENVVALUE04 = Beta('SIGMA_STAR_ENVVALUE04', 1, 1.0e-5, None, 0)

delta_1 = Beta('delta_1', 0.1, 1.0e-5, None, 1)
delta_2 = Beta('delta_2', 0.2, 1.0e-5, None, 0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

ENVVALUE01_tau_1 = (tau_1 - MODEL_ENVVALUE01) / SIGMA_STAR_ENVVALUE01
ENVVALUE01_tau_2 = (tau_2 - MODEL_ENVVALUE01) / SIGMA_STAR_ENVVALUE01
ENVVALUE01_tau_3 = (tau_3 - MODEL_ENVVALUE01) / SIGMA_STAR_ENVVALUE01
ENVVALUE01_tau_4 = (tau_4 - MODEL_ENVVALUE01) / SIGMA_STAR_ENVVALUE01
ENVVALUE01_tau_5 = (tau_5 - MODEL_ENVVALUE01) / SIGMA_STAR_ENVVALUE01

ENVVALUE02_tau_1 = (tau_1 - MODEL_ENVVALUE02) / SIGMA_STAR_ENVVALUE02
ENVVALUE02_tau_2 = (tau_2 - MODEL_ENVVALUE02) / SIGMA_STAR_ENVVALUE02
ENVVALUE02_tau_3 = (tau_3 - MODEL_ENVVALUE02) / SIGMA_STAR_ENVVALUE02
ENVVALUE02_tau_4 = (tau_4 - MODEL_ENVVALUE02) / SIGMA_STAR_ENVVALUE02
ENVVALUE02_tau_5 = (tau_5 - MODEL_ENVVALUE02) / SIGMA_STAR_ENVVALUE02

ENVVALUE03_tau_1 = (tau_1 - MODEL_ENVVALUE03) / SIGMA_STAR_ENVVALUE03
ENVVALUE03_tau_2 = (tau_2 - MODEL_ENVVALUE03) / SIGMA_STAR_ENVVALUE03
ENVVALUE03_tau_3 = (tau_3 - MODEL_ENVVALUE03) / SIGMA_STAR_ENVVALUE03
ENVVALUE03_tau_4 = (tau_4 - MODEL_ENVVALUE03) / SIGMA_STAR_ENVVALUE03
ENVVALUE03_tau_5 = (tau_5 - MODEL_ENVVALUE03) / SIGMA_STAR_ENVVALUE03

ENVVALUE04_tau_1 = (tau_1 - MODEL_ENVVALUE04) / SIGMA_STAR_ENVVALUE04
ENVVALUE04_tau_2 = (tau_2 - MODEL_ENVVALUE04) / SIGMA_STAR_ENVVALUE04
ENVVALUE04_tau_3 = (tau_3 - MODEL_ENVVALUE04) / SIGMA_STAR_ENVVALUE04
ENVVALUE04_tau_4 = (tau_4 - MODEL_ENVVALUE04) / SIGMA_STAR_ENVVALUE04
ENVVALUE04_tau_5 = (tau_5 - MODEL_ENVVALUE04) / SIGMA_STAR_ENVVALUE04


# Likert scale with M = 6 measurements
IndENVVALUE01 = {
    1: bioNormalCdf(ENVVALUE01_tau_1),
    2: bioNormalCdf(ENVVALUE01_tau_2) - bioNormalCdf(ENVVALUE01_tau_1),
    3: bioNormalCdf(ENVVALUE01_tau_3) - bioNormalCdf(ENVVALUE01_tau_2),
    4: bioNormalCdf(ENVVALUE01_tau_4) - bioNormalCdf(ENVVALUE01_tau_3),
    5: bioNormalCdf(ENVVALUE01_tau_5) - bioNormalCdf(ENVVALUE01_tau_4),
    6: 1 - bioNormalCdf(ENVVALUE01_tau_5)
}
P_ENVVALUE01 = Elem(IndENVVALUE01, envvalues1)

IndENVVALUE02 = {
    1: bioNormalCdf(ENVVALUE02_tau_1),
    2: bioNormalCdf(ENVVALUE02_tau_2) - bioNormalCdf(ENVVALUE02_tau_1),
    3: bioNormalCdf(ENVVALUE02_tau_3) - bioNormalCdf(ENVVALUE02_tau_2),
    4: bioNormalCdf(ENVVALUE02_tau_4) - bioNormalCdf(ENVVALUE02_tau_3),
    5: bioNormalCdf(ENVVALUE02_tau_5) - bioNormalCdf(ENVVALUE02_tau_4),
    6: 1 - bioNormalCdf(ENVVALUE02_tau_5)
}
P_ENVVALUE02 = Elem(IndENVVALUE02, envvalues2)

IndENVVALUE03 = {
    1: bioNormalCdf(ENVVALUE03_tau_1),
    2: bioNormalCdf(ENVVALUE03_tau_2) - bioNormalCdf(ENVVALUE03_tau_1),
    3: bioNormalCdf(ENVVALUE03_tau_3) - bioNormalCdf(ENVVALUE03_tau_2),
    4: bioNormalCdf(ENVVALUE03_tau_4) - bioNormalCdf(ENVVALUE03_tau_3),
    5: bioNormalCdf(ENVVALUE03_tau_5) - bioNormalCdf(ENVVALUE03_tau_4),
    6: 1 - bioNormalCdf(ENVVALUE03_tau_5)
}
P_ENVVALUE03 = Elem(IndENVVALUE03, envvalues3)

IndENVVALUE04 = {
    1: bioNormalCdf(ENVVALUE04_tau_1),
    2: bioNormalCdf(ENVVALUE04_tau_2) - bioNormalCdf(ENVVALUE04_tau_1),
    3: bioNormalCdf(ENVVALUE04_tau_3) - bioNormalCdf(ENVVALUE04_tau_2),
    4: bioNormalCdf(ENVVALUE04_tau_4) - bioNormalCdf(ENVVALUE04_tau_3),
    5: bioNormalCdf(ENVVALUE04_tau_5) - bioNormalCdf(ENVVALUE04_tau_4),
    6: 1 - bioNormalCdf(ENVVALUE04_tau_5)
}
P_ENVVALUE04 = Elem(IndENVVALUE04, envvalues4)


In [15]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
# ASC_CAR_S = Beta('ASC_CAR_S', 1, None, None, 0)
# ASC_CAR_RND = ASC_CAR + ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_ANTI')

ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
# ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 1, None, None, 0)
# ASC_TRANSIT_RND = ASC_TRANSIT + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_RH = Beta('ASC_RH', 0, None, None, 0)
# ASC_RH_S = Beta('ASC_RH_S', 1, None, None, 0)
# ASC_RH_RND = ASC_RH + ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_ANTI')

# ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
# ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
# ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

# ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
# ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
# ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

# ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
# ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
# ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

# ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
# ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
# ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

B_CARTIME = Beta('B_CARTIME', 0, None, None, 0)
B_TRANSITTIME = Beta('B_TRANSITTIME', 0, None, None, 0)
B_RHTIME = Beta('B_RHTIME', 0, None, None, 0)
B_WALKTIME = Beta('B_WALKTIME', 0, None, None, 0)
B_PERSBIKETIME = Beta('B_PERSBIKETIME', 0, None, None, 0)

B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
# B_DKBIKETIME = Beta('B_DKBIKETIME', 0, None, None, 0)
B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_COST = Beta('B_COST', 0, None, None, 0)
B_COST_PRIME = Beta('B_COST_PRIME', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_SQ = Beta('B_AGE_SQ', 0, None, None, 0)
B_AGE_SCTRANSIT_SQ = Beta('B_AGE_SCTRANSIT_SQ', 0, None, None, 0)
B_GENDER_F = Beta('B_GENDER_F', 0, None, None, 0)

B_AGE_MORE_35 = Beta('B_AGE_MORE_35', 0, None, None, 0)
B_CHILD_IN_HH = Beta('B_CHILD_IN_HH', 0, None, None, 0)
B_IS_WHITE = Beta('B_IS_WHITE', 0, None, None, 0)
B_FULLTIME_EMPLOY = Beta('B_FULLTIME_EMPLOY', 0, None, None, 0)
B_HHINCOME_MORE_75k = Beta('B_HHINCOME_MORE_75k', 0, None, None, 0)

B_RISK_BIKE = Beta('B_RISK_BIKE', 0, None, None, 0)
B_RISK_SCTRANSIT = Beta('B_RISK_SCTRANSIT', 0, None, None, 0)
B_RISK_TRANSIT = Beta('B_RISK_TRANSIT', 0, None, None, 0)
B_RISK_RH = Beta('B_RISK_RH', 0, None, None, 0)
B_RISK_PERSBIKE = Beta('B_RISK_PERSBIKE', 0, None, None, 0)
B_RISK_WALK = Beta('B_RISK_WALK', 0, None, None, 0)
B_RISK_SCOOTER = Beta('B_RISK_SCOOTER', 0, None, None, 0)

B_RISK_MICROMOBILITY = Beta('B_RISK_MICROMOBILITY', 0, None, None, 0)
B_RISK_ACTIVETRANSP = Beta('B_RISK_ACTIVETRANSP', 0, None, None, 0)

B_NORM_MICROMOBILITY_SCOOTER = Beta('B_NORM_MICRO_SCOOT', 0, None, None, 0)
B_NORM_MICROMOBILITY_BIKE = Beta('B_NORM_MICRO_BIKE', 0, None, None, 0)

B_ENVVALUE_MICROMOBILITY_SCOOTER = Beta('B_ENVVALUE_MICRO_SCOOT', 0, None, None, 0)
B_ENVVALUE_MICROMOBILITY_BIKE = Beta('B_ENVVALUE_MICRO_BIKE', 0, None, None, 0)
B_ENVVALUE_MICROMOBILITY = Beta('B_ENVVALUE_MICROMOBILITY', 0, None, None, 0)

In [16]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
ASC_CAR_S = Beta('ASC_CAR_S', 1, None, None, 0)
ASC_CAR_RND = ASC_CAR + ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_ANTI')

ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 1, None, None, 0)
ASC_TRANSIT_RND = ASC_TRANSIT + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_RH = Beta('ASC_RH', 0, None, None, 0)
ASC_RH_S = Beta('ASC_RH_S', 1, None, None, 0)
ASC_RH_RND = ASC_RH + ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_ANTI')

ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_WALK_S = Beta('ASC_WALK_S', 1, None, None, 0 )
ASC_WALK_RND = ASC_WALK + ASC_WALK_S * bioDraws('ASC_WALK_RND', 'NORMAL_ANTI')

# BIKE mode parameters
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_BIKE_S = Beta('ASC_BIKE_S', 1, None, None, 0)
ASC_BIKE_RND = ASC_BIKE + ASC_BIKE_S * bioDraws('ASC_BIKE_RND', 'NORMAL_ANTI')

# SCOOTER mode parameters
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

# DKBIKE mode parameters
ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

# DLBIKE mode parameters
ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

# SCTRANSIT mode parameters
ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')


In [17]:
# # Nested logit terms, groupings = micromobility, conventional, active transportation, other
# draw_micromobility = bioDraws('ASC_Micromobility', 'NORMAL_HALTON3')
# draw_activetransp = bioDraws('ASC_ActiveTransp', 'NORMAL_HALTON3')
# ASC_ACTIVETRANSP = Beta('ASC_ACTIVE TRANSP', 0, None, None, 0)
# ASC_MICROMOBILITY = Beta('ASC_MICROMOBILITY', 0, None, None, 0)


# ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
# ASC_WALK_S = Beta('ASC_WALK_S', 1, None, None, 0)
# ASC_WALK_RND = ASC_WALK + ASC_ACTIVETRANSP * draw_activetransp

# ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
# ASC_BIKE_S = Beta('ASC_BIKE_S', 1, None, None, 0)
# ASC_BIKE_RND = ASC_BIKE + ASC_ACTIVETRANSP * draw_activetransp

# ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
# ASC_SCOOTER_RND = ASC_SCOOTER + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

# ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
# ASC_DLBIKE_RND = ASC_DLBIKE + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp 

# ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
# ASC_DKBIKE_RND = ASC_DKBIKE + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

# ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
# ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

In [18]:
B_CONV_NONHOMEBASE = Beta('B_CONV_NONHOMEBASE', 0, None, None, 0)
B_MICRO_NONHOMEBASE = Beta('B_MICRO_NONHOMEBASE', 0, None, None, 0)

In [21]:
# Utility functions
V1 = (ASC_CAR_RND
      + B_CARTIME*cartime
      )

V2 = (ASC_TRANSIT_RND
      + B_TRANSITTIME*transittime
      # + B_RISK_TRANSIT*PERCEIVEDRISK
      + B_CONV_NONHOMEBASE * trippurp_NHB
      )

V3 = (ASC_RH_RND
      + B_RHTIME*rdtime
      # + B_RISK_RH*PERCEIVEDRISK
      + B_CONV_NONHOMEBASE * trippurp_NHB
)

V4 = (ASC_WALK_RND
      + B_WALKTIME*walktime
      # + B_RISK_WALK*PERCEIVEDRISK
#       + B_RISK_ACTIVETRANSP*PERCEIVEDRISK
      + B_CONV_NONHOMEBASE * trippurp_NHB
)

V5 = (ASC_BIKE_RND
      + B_PERSBIKETIME*biketime
      # + B_RISK_PERSBIKE*PERCEIVEDRISK
# #       + B_RISK_ACTIVETRANSP*PERCEIVEDRISK
      + B_CONV_NONHOMEBASE * trippurp_NHB
      
)

V7 = (ASC_SCOOTER_RND   
      #+ ASC_SCOOTER_PRIME*AVtech
      + B_SCOOTERTIME*sctime
      #+ B_SCOOTERTIME_PRIME*sctime*AVtech
      + B_COST*sccost_adj
      #+ B_COST_PRIME*sccost_adj*AVtech
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech
      + B_MICRO_NONHOMEBASE * trippurp_NHB
      # + B_RISK_SCOOTER*PERCEIVEDRISK
#       + B_RISK_MICROMOBILITY*PERCEIVEDRISK
      + B_NORM_MICROMOBILITY_SCOOTER*SOCIALNORM
      # + B_ENVVALUE_MICROMOBILITY * ENVVALUE
      
      )
V8 = (ASC_DLBIKE_RND
        + B_BIKETIME*dltime
        + B_COST*dlcost_adj
        + B_ACCESS*DLAW
        + B_MICRO_NONHOMEBASE * trippurp_NHB
#         + B_RISK_MICROMOBILITY*PERCEIVEDRISK
        + B_NORM_MICROMOBILITY_BIKE*SOCIALNORM
      # + B_RISK_BIKE*PERCEIVEDRISK
      #    + B_ENVVALUE_MICROMOBILITY * ENVVALUE
       
    )
V9 = (ASC_DKBIKE_RND 
      + B_BIKETIME*dbtime 
      + B_COST*dbcost_adj
      + B_ACCESS*DBAW
      + B_DROP*DBDW
      + B_MICRO_NONHOMEBASE * trippurp_NHB
      # + B_RISK_BIKE*PERCEIVEDRISK
      + B_NORM_MICROMOBILITY_BIKE*SOCIALNORM
      #    + B_ENVVALUE_MICROMOBILITY * ENVVALUE
   
    
    )
V10 = (ASC_SCTRANSIT_RND
       + B_SCTRANSITTIME*sttotime
       + B_COST*sttocost_adj 
       + B_ACCESS*STAW*(1-AVtech)
       + B_WAITAV*STAV*AVtech 
      #  + B_RISK_SCTRANSIT*PERCEIVEDRISK
       + B_NORM_MICROMOBILITY_SCOOTER*SOCIALNORM
      + B_MICRO_NONHOMEBASE * trippurp_NHB
      #    + B_ENVVALUE_MICROMOBILITY * ENVVALUE
     
)

In [22]:
V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5, 7: V7, 8: V8, 9: V9, 10: V10}
av = {1: car_av, 2: transit_av, 3: rd_av, 4: walk_av, 5: bike_av, 
      7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}
      

condprob = models.logit(V, av, choice)
condlike = (
    P_Norm01 *
    P_Norm02 *
    P_Norm03 *
    P_Norm04 *
    P_Norm05 *
    P_Norm06 *
    condprob
)
condlikeindiv = PanelLikelihoodTrajectory(condprob)
logprob = log(MonteCarlo(condlikeindiv))

biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 250, numberOfThreads = 100)

# biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = "Mixed Logit with Social Norm, 6 indicators - 250 draws"
# biogeme.modelName = "HybridChoiceNestedLogit"

results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(f"Estimated betas: {len(results.data.betaValues)}")
print(f"Final log likelihood: {results.data.logLike:.3f}")
print(pandasResults.round(2))

The use of argument numberOfThreads in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_threads in section MultiThreadingof the .toml parameter file. The default file name is biogeme.toml
The use of argument numberOfDraws in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_draws in section MonteCarloof the .toml parameter file. The default file name is biogeme.toml


In [ ]:
# corr = calculate_correlation(
#     nests, results, alternative_names = {1: 'Car', 2: 'Transit', 3: 'Ridehailing', 4: 'Walk', 5: 'Personal Bike',
#                                          7: 'Scooter', 8: 'Dockless Bike', 9: 'Docked Bike', 10: 'Scooter + Transit'}
# )
# print(corr)